# Digging deeper into weights, GPT-2 huggingface transformer

In [ ]:
from transformers import GPT2LMHeadModel

In [ ]:
model_hf = GPT2LMHeadModel.from_pretrained("gpt2")
sd_hf = model_hf.state_dict() 


for k, v in sd_hf.items():
	print(k, v.shape)
	

In [ ]:
# calculating total number of parameters
total_params = 0
for k, v in sd_hf.items():
	total_params += v.numel()
print(f"Total number of parameters in millions: {total_params / 1e6:.2f}")
print(f"Total number of parameters in billions: {total_params / 1e9:.2f}")

In [ ]:
sd_hf['transformer.h.0.ln_1.weight'][:10]

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

plt.imshow(sd_hf["transformer.wpe.weight"], cmap="gray")

In [ ]:
sd_hf["transformer.wpe.weight"].shape

In [ ]:
plt.plot(sd_hf["transformer.wpe.weight"][:, 0])
plt.plot(sd_hf["transformer.wpe.weight"][:, 1])
plt.plot(sd_hf["transformer.wpe.weight"][:, 2])

In [ ]:
from transformers import pipeline, set_seed
generator = pipeline('text-generation', model='gpt2-xl')
set_seed(42)
generated_texts = generator(
	"",
	max_length=100,
	num_return_sequences=5
)

for i, output in enumerate(generated_texts, start=1):
	print(f"Sequence {i}:\n{output['generated_text']}\n")


In [ ]:
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer, pipeline, set_seed

# pick the MPS device if available, else fall back to CPU
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")

# load model+tokenizer, move model to mps
model = GPT2LMHeadModel.from_pretrained("gpt2-xl").to(device)
tokenizer = GPT2Tokenizer.from_pretrained("gpt2-xl")


In [ ]:
model.device

# Synthesizing/Generating text with GPT-2

In [1]:
import torch
import time
from transformers import GPT2LMHeadModel, GPT2Tokenizer, pipeline, set_seed

# pick the MPS device if available, else fall back to CPU
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")

# load model+tokenizer, move model to mps
model = GPT2LMHeadModel.from_pretrained("gpt2-xl").to(device)
tokenizer = GPT2Tokenizer.from_pretrained("gpt2-xl")

# print where the model lives
print("model device:", next(model.parameters()).device)

# run a quick timing check
inp = tokenizer("Hello world", return_tensors="pt").to(model.device)
start = time.time()
_ = model(**inp)
print(f"single forward pass: {time.time() - start:.3f}s")

/Users/shivam13juna/Documents/virtual_envs/appy/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/shivam13juna/Documents/virtual_envs/appy/lib/python3.9/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


model device: mps:0
single forward pass: 0.196s


In [3]:

# build your pipeline using the already‑moved model
generator = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    device='mps'               # None tells HF to respect model.device
)

generated_texts = generator(
	"Why do countries go to war?",
	max_length=100,
	num_return_sequences=5
)

for i, output in enumerate(generated_texts, start=1):
	print(f"Sequence {i}:\n{output['generated_text']}\n")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Sequence 1:
Why do countries go to war? A common reason is that someone's family or friends have been killed. If someone's family is killed, they are more motivated to send troops into the war. For many countries that war has been caused by external aggression, e.g. in Iraq or Libya. The U.S. and Britain have launched wars on other countries to force a change of regime to that country to support the American interests. There were many other reasons (like money), but let's

Sequence 2:
Why do countries go to war? It's a question I asked in the 1970s and the answer was a military answer. Now it's more of an economic question," says Smejkal, co-author of The War We Never Fought: How the US Waged the Korean War, Vietnam, and Iraq.

Smejkal is the author of three books on World War II in Korea, the subject of a Pulitzer Prize. In his book he traces America's

Sequence 3:
Why do countries go to war? Who really benefits or suffers? Is it just a matter of the cost, or is there more involved — 

In [ ]:
import torch
import time

# print where the model lives
print("model device:", next(model.parameters()).device)

# run a quick timing check
inp = tokenizer("Hello world", return_tensors="pt").to(model.device)
start = time.time()
_ = model(**inp)
print(f"single forward pass: {time.time() - start:.3f}s")

In [ ]:
import torch
assert torch.backends.mps.is_available(), "MPS not available on this machine"